In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

load_dotenv(override=True)

/home/jeal/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Pre: Getting chunks

In [8]:
# Load from hub
ds_questions = load_dataset(
    "jealk/dk_retrieval_benchmark",
    "generated_questions",
    split="train",
    download_mode="force_redownload",
)

df_questions = ds_questions.to_pandas()
df_questions.head()

Generating train split: 100%|██████████| 200/200 [00:00<00:00, 89164.63 examples/s]


,title_vejledning,chunk_text,url,generated_question
0,Vejledning om regulering af satser fra 1. janu...,Vejledning om regulering af satser fra 1. janu...,https://www.retsinformation.dk/eli/retsinfo/20...,Hvem fastsætter reguleringen af satserne for a...
1,Vejledning om regulering af satser fra 1. janu...,Vejledning om regulering af satser fra 1. janu...,https://www.retsinformation.dk/eli/retsinfo/20...,Med hvilken procent vil satserne for arbejdssk...
2,Vejledning om regulering af satser fra 1. janu...,Tilpasningsprocenten er ved Finansministeriets...,https://www.retsinformation.dk/eli/retsinfo/20...,Hvad er tilpasningsprocenten for finansåret 20...
3,Vejledning om regulering af satser fra 1. janu...,Tilpasningsprocenten er ved Finansministeriets...,https://www.retsinformation.dk/eli/retsinfo/20...,Fra hvilken dato træder de nye satser i lov om...
4,Vejledning om regulering af satser fra 1. janu...,1541 af 12. december 2023 fastsættes løbende e...,https://www.retsinformation.dk/eli/retsinfo/20...,"Fra hvilken dato vil arbejdsskader, der sker, ..."


In [9]:
# Rename and drop columns to fit promptlayer evaluation
# Drop generated_question
df_questions = df_questions.drop(columns=["generated_question"])
# Delete every second row
df_questions = df_questions[::2]
df_questions.head()

,title_vejledning,chunk_text,url
0,Vejledning om regulering af satser fra 1. janu...,Vejledning om regulering af satser fra 1. janu...,https://www.retsinformation.dk/eli/retsinfo/20...
2,Vejledning om regulering af satser fra 1. janu...,Tilpasningsprocenten er ved Finansministeriets...,https://www.retsinformation.dk/eli/retsinfo/20...
4,Vejledning om regulering af satser fra 1. janu...,1541 af 12. december 2023 fastsættes løbende e...,https://www.retsinformation.dk/eli/retsinfo/20...
6,Vejledning om regulering af satser fra 1. janu...,"og det maksimale årslønsbeløb, der var gældend...",https://www.retsinformation.dk/eli/retsinfo/20...
8,Vejledning om regulering af satser fra 1. janu...,Satser for arbejdsskader indtruffet 1. juli 20...,https://www.retsinformation.dk/eli/retsinfo/20...


In [10]:
# Save chunk_text as a list of strings
chunk_list = df_questions["chunk_text"].apply(lambda x: [x])
chunk_list[0]

['Vejledning om regulering af satser fra 1. januar 2024 efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde\nIndledning\nEfter lov om arbejdsskadesikring, jf. lovbekendtgørelse nr. 1186 af 19. august 2022 med de ændringer, der følger af lov nr. 1541 af 12. december 2023, og lov om sikring mod følger af arbejdsskade, jf. lovbekendtgørelse nr. 943 af 16. oktober 2000, skal der med virkning fra 1. januar 2024 efter indstilling fra bestyrelsen for Arbejdsmarkedets Erhvervssikring ske regulering af lovens årslønsbeløb, godtgørelsesbeløb, overgangsbeløb samt løbende erstatninger.\nReguleringen af satserne fastsættes af Arbejdstilsynets direktør efter bemyndigelse fra beskæftigelsesministeren.\nSatser efter loven reguleres med 2 procent tillagt tilpasningsprocenten for finansåret 2024 (jf. lov om en satsreguleringsprocent).']

## Pre, getting documents

In [13]:
# Load from hub
ds_vejledninger = load_dataset(
    "jealk/dk_retrieval_benchmark",
    "retsinformation",
    split="train",
    download_mode="force_redownload",
)

Generating train split: 100%|██████████| 433/433 [00:00<00:00, 1262.34 examples/s]


In [14]:
# Create pandas dataframe from the dataset using the huggingface datasets library
df_vejledninger = ds_vejledninger.to_pandas()
df_vejledninger.head()

,url,title,html_content,text_content
0,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om regulering af satser fra 1. janu...,"<div class=""document-content"" id=""restylingRoo...",Vejledning om regulering af satser fra 1. janu...
1,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om satser i 2024 for betaling af ud...,"<div class=""document-content"" id=""restylingRoo...",Vejledning om satser i 2024 for betaling af ud...
2,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om obligatorisk selvbooking af jobs...,"<div class=""document-content"" id=""restylingRoo...",Vejledning om obligatorisk selvbooking af jobs...
3,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning til bekendtgørelse om tilskud til s...,"<div class=""document-content"" id=""restylingRoo...",Vejledning til bekendtgørelse om tilskud til s...
4,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om fleksløntilskud m.v.,"<div class=""document-content"" id=""restylingRoo...",Vejledning om fleksløntilskud m.v.\n1.Indledni...


## Function overview

- Step 0: Chunking text
    - Include or not?
- Step 1: Filter chunks
    - W. Textdescriptives
    - W. LLM call, egnet til spørgsmål?
- Step 2: Generate questions:
    - Using LLamaIndex
- Step 3: Filter generated questions
    - (Text descriptives for long texts)
    - LLM call: Is the answer found in chunk?
    - LLM call: Is the answer clear and in a natural language?
- Step 4: Update chunk-question table
    - Embed chunks, embed questions (Local Vector DB)
    - Use vector search to identify top 10 matches
    - (Optional, Rerank)
    - Filtering: Flag query/chunks where intended match is not in Top @10
    - If question/chunk not @1
        - Use LLM to check any question/chunk scored > than "real" match
        - Update Match Matrix if OK
    - If Delta simililarity score from 'real match' to other top @10 is < threshold:
        - Use LLM to check question/chunk
        - Update Match Matrix if OK
- Step 5: Convert to BEIR format

# Step 0

In [19]:
from typing import List, Dict, Any
from llama_index import Document


def create_documents(text: List[str], metadata: List[Dict[str, Any]]) -> List[Document]:
    """Create a list of llama_index documents from a list of strings and a list of dictionaries

    Args:
    text: A list of strings containing the text of the documents, eg. ["Vejledning om ...", "..."]
    metadata: A list of dictionaries containing one or multiple metadata, eg. [{"title": "Example 1", "source": "website_url"}, {...}]

    Returns:
    A list of llama_index documents
    """
    documents = [
        Document(text=content, metadata=meta) for content, meta in zip(text, metadata)
    ]
    return documents

In [21]:
# Create documents from the dataframe
llama_documents = create_documents(
    df_vejledninger["text_content"],
    df_vejledninger[["title", "url"]].to_dict(orient="records"),
)

In [36]:
from llama_index.node_parser import SentenceSplitter
from transformers import AutoTokenizer
from llama_index.core.schema import TextNode


def document_splitter(
    documents: List[Document],
    chunk_size: int = 512,
    tokenizer=AutoTokenizer.from_pretrained("intfloat/e5-base-v2"),
) -> List[TextNode]:
    """Split a list of llama_index documents into nodes

    Args:
    documents: A list of llama_index documents
    chunk_size: An integer defining the maximum number of tokens in each node
    tokenizer: A tokenizer from the Hugging Face transformers library

    Returns:
    A list of nodes, consisting of text, metadata, embeddings and node-relations
    """
    node_parser = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=0,
        secondary_chunking_regex=str(["\n"]),
        paragraph_separator=str(["\n\n"]),
        tokenizer=tokenizer.tokenize,
    )
    nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)
    return nodes


nodes_vejledninger = document_splitter(llama_documents)

# Step 1

# Step 2

In [ ]:
from llama_index.prompts import PromptTemplate

# Define your custom prompt template in Danish
qa_sagsbehandler_tmlp = """ Nedenfor er et uddrag (kontekst) fra en længere tekst:
---------------------
{context_str}
---------------------
Givet ovenstående uddrag og ingen forudgående viden, er din opgave at generere spørgsmål til teksten.
Spørgsmålet skal indeholde specifik kontekst, således at spørgsmålet kan besvares uden tvetydighed udenfor uddraget. 
Du er en erfaren sagsbehandler, og din opgave er at stille præcis {num_questions_per_chunk} spørgsmål, som kan besvares i uddraget.
Spørgsmålene skal være af forskellig karakter og dække teksten bredt, men stilles i et sprog som en borger uden juridisk ekspertise kan forstå.
Svaret til spørgsmålet, skal kunne findes i ovenstående uddrag.

Eksempel på et spørgsmål der ikke har en specifik kontekst: 
"Hvilket dokument har den nye vejledning erstattet, og hvornår blev det tidligere dokument udsendt?", da spørgsmålet er tvetydigt og vil have flere svarmuligheder afhængigt af hvilken vejledning og dokument der refereres til.

Eksempel på et godt spørgsmål, som kan besvares uden kendskab til uddraget:
"Hvor meget vil godtgørelsen for et varigt mén være, hvis det er vurderet til 100%?"

"""

qa_sagsbehandler_tmlp = PromptTemplate(qa_sagsbehandler_tmlp)

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.llms import OpenAI

# define LLM
llm_oai = OpenAI(temperature=0.0, model="gpt-4-0125-preview")

qa_dataset = generate_qa_embedding_pairs(
    qa_generate_prompt_tmpl=qa_sagsbehandler_tmlp,
    llm=llm_oai,
    nodes=nodes_vejledninger_filtered[0:10],
    num_questions_per_chunk=2,
)

In [ ]:
# As an experienced software engineer and following best practice, write a function that takes a list of strings and